In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../data/processed/traffic_volume_2018_plus_feature_engineered.csv")

# Quick look
df.head()

# Basic info
df.info()

# Check for nulls
df.isnull().sum()

/var/folders/px/j7p2_nnn6h339lc16xg05qbw0000gn/T/ipykernel_53826/3481257512.py:1: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/traffic_volume_2018_plus_feature_engineered.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528625 entries, 0 to 528624
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   requestid          528625 non-null  int64 
 1   boro               528625 non-null  object
 2   year               528625 non-null  int64 
 3   month              528625 non-null  int64 
 4   day                528625 non-null  int64 
 5   hour               528625 non-null  int64 
 6   minute             528625 non-null  int64 
 7   vol                528625 non-null  int64 
 8   segmentid          528625 non-null  int64 
 9   wktgeom            528625 non-null  object
 10  street             528625 non-null  object
 11  fromst             528625 non-null  object
 12  tost               528625 non-null  object
 13  direction          528625 non-null  object
 14  timestamp          528625 non-null  object
 15  dayofweek          528625 non-null  object
 16  is_weekend         5

requestid                 0
boro                      0
year                      0
month                     0
day                       0
hour                      0
minute                    0
vol                       0
segmentid                 0
wktgeom                   0
street                    0
fromst                    0
tost                      0
direction                 0
timestamp                 0
dayofweek                 0
is_weekend                0
is_rush_hour              0
is_holiday                0
holiday_name         510577
holiday_type         510577
is_holiday_period         0
dtype: int64

In [ ]:
# Line Plot: Volume Over Time
plt.figure(figsize=(14, 6))
sns.lineplot(data=df, x='timestamp', y='vol')
plt.title("Traffic Volume Over Time")
plt.xlabel("Time")
plt.ylabel("Volume")
plt.tight_layout()
plt.show()